<a href="https://colab.research.google.com/github/cvcdrew/WeatherWise-Andrew/blob/main/Copy_of_starter_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🌦️ WeatherWise – Starter Notebook



In [ ]:
# 🧪 Optional packages — uncomment if needed in Colab or JupyterHub
!pip install fetch-my-weather
!pip install hands-on-ai
!pip install pyinputplus
!pip install customtkinter
!pip install requests
!pip install timezonefinder geopy
!pip install langdetect


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 83.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.1/66.1 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.5/80.5 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 459.8/459.8 kB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 104.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.8/454.8 kB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 103.6 MB/s eta 0:00:00
  Attempting uninstall: fastcore
    Found existing

In [ ]:
import os

os.environ['HANDS_ON_AI_SERVER'] = 'http://ollama.serveur.au'
os.environ['HANDS_ON_AI_MODEL'] = 'granite3.2'
os.environ['HANDS_ON_AI_API_KEY'] = input('Enter your API key: ')

Enter your API key: 050d8193513b67975f932a4bb3b864de


## 📦 Setup and Configuration
Import required packages and setup environment.

In [ ]:
import requests
import matplotlib.pyplot as plt
import pyinputplus as pyip
import pandas as pd
from datetime import datetime, timedelta

import seaborn as sns
# ✅ Import after installing (if needed)
from fetch_my_weather import get_weather
from hands_on_ai.chat import get_response

# Add any other setup code here

## 🌤️ Weather Data Functions

In [ ]:
import requests
from typing import Dict

# === Configuration ===
API_KEY = "your_api_key_here"  # Replace with your actual API key
BASE_URL = "https://api.openweathermap.org/data/2.5/forecast"
INTERVALS_PER_DAY = 8  # 3-hour intervals

# === Core Weather Functions ===

def build_weather_query_params(location: str, forecast_days: int, units: str) -> Dict[str, str]:
    """Build query parameters for the weather API request."""
    return {
        "q": location,
        "cnt": forecast_days * INTERVALS_PER_DAY,
        "units": units,  # 'metric', 'imperial', or 'standard'
        "appid": API_KEY
    }

def fetch_weather_data(params: dict) -> dict:
    """Fetch weather data from the API."""
    try:
        response = requests.get(BASE_URL, params=params)
        response.raise_for_status()
        return response.json()
    except requests.RequestException as e:
        print(f"[ERROR] Failed to fetch weather data: {e}")
        return {}

def get_weather_data(location: str, forecast_days: int = 5, units: str = "metric") -> dict:
    """Main function to get weather data for a given location and forecast period."""
    if units not in ["standard", "metric", "imperial"]:
        print(f"[WARNING] Invalid units '{units}' provided. Defaulting to 'metric'.")
        units = "metric"

    params = build_weather_query_params(location, forecast_days, units)
    return fetch_weather_data(params)

# === Display Logic ===

def display_weather(data: dict, units: str):
    """Format and display the weather data."""
    if not data or "list" not in data:
        print("No weather data available.")
        return

    city = data.get("city", {}).get("name", "Unknown location")
    country = data.get("city", {}).get("country", "")
    print(f"\n📍 Weather Forecast for {city}, {country}:\n")

    temp_unit = "°C" if units == "metric" else "°F" if units == "imperial" else "K"
    wind_unit = "m/s" if units in ["metric", "standard"] else "mph"

    for forecast in data["list"]:
        dt_txt = forecast["dt_txt"]
        temp = forecast["main"]["temp"]
        weather = forecast["weather"][0]["description"].capitalize()
        wind = forecast["wind"]["speed"]

        print(f"{dt_txt} | 🌡 Temp: {temp}{temp_unit} | 🌬 Wind: {wind} {wind_unit} | 🌥 {weather}")

# === CLI Interface ===

def main():
    print("=== Weather Forecast CLI ===")
    location = input("Enter location (e.g., London): ")
    try:
        days = int(input("Enter number of forecast days (1-5): "))
        if not (1 <= days <= 5):
            raise ValueError
    except ValueError:
        print("[ERROR] Please enter a valid number of days (1–5).")
        return

    units = input("Enter units (metric / imperial / standard): ").strip().lower()

    data = get_weather_data(location, days, units)
    display_weather(data, units)

# === Run Program ===

if __name__ == "__main__":
    main()


## 📊 Visualisation Functions

In [ ]:
# === Imports ===
import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates
from datetime import timedelta
from geopy.geocoders import Nominatim
from timezonefinder import TimezoneFinder
from zoneinfo import ZoneInfo  # Requires Python 3.9+

# === Configuration ===
API_KEY = "your_api_key_here"  # Replace with your OpenWeatherMap API key
BASE_URL = "https://api.openweathermap.org/data/2.5/forecast"
INTERVALS_PER_DAY = 8

# === Timezone Utilities ===

def get_lat_lon_from_city(city_name):
    geolocator = Nominatim(user_agent="weather-app")
    location = geolocator.geocode(city_name)
    if location:
        return location.latitude, location.longitude
    else:
        raise ValueError(f"Could not find location for city: {city_name}")

def get_timezone_from_latlon(lat, lon):
    tf = TimezoneFinder()
    timezone = tf.timezone_at(lat=lat, lng=lon)
    if timezone:
        return timezone
    else:
        raise ValueError("Could not determine timezone.")

def get_timezone_for_city(city_name):
    lat, lon = get_lat_lon_from_city(city_name)
    return get_timezone_from_latlon(lat, lon)

def convert_to_local_time(dates_utc, timezone_str):
    return pd.to_datetime(dates_utc).dt.tz_localize('UTC').dt.tz_convert(ZoneInfo(timezone_str))

# === Weather Data Fetching ===

def build_weather_query_params(location: str, forecast_days: int, units: str):
    return {
        "q": location,
        "cnt": forecast_days * INTERVALS_PER_DAY,
        "units": units,
        "appid": API_KEY
    }

def fetch_weather_data(params: dict):
    try:
        response = requests.get(BASE_URL, params=params)
        response.raise_for_status()
        return response.json()
    except requests.RequestException as e:
        print(f"[ERROR] Failed to fetch weather data: {e}")
        return {}

def get_weather_data(location: str, forecast_days: int = 5, units: str = "metric"):
    if units not in ["metric", "imperial", "standard"]:
        print(f"[WARNING] Invalid units '{units}' provided. Defaulting to 'metric'.")
        units = "metric"

    params = build_weather_query_params(location, forecast_days, units)
    return fetch_weather_data(params)

# === Preprocessing and Plotting ===

def preprocess_weather_data(dates, values, max_points=200, timezone="UTC"):
    df = pd.DataFrame({
        'Date': pd.to_datetime(dates),
        'Value': values
    })
    df['Date'] = convert_to_local_time(df['Date'], timezone)
    if len(df) > max_points:
        df = df.iloc[::len(df) // max_points]
    return df

def format_time_axis(ax, dates):
    if isinstance(dates, list):
        dates = pd.to_datetime(dates)

    time_span = dates.max() - dates.min()

    if time_span <= timedelta(hours=12):
        ax.xaxis.set_major_locator(mdates.HourLocator(interval=1))
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
    elif time_span <= timedelta(days=1):
        ax.xaxis.set_major_locator(mdates.HourLocator(interval=3))
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
    elif time_span <= timedelta(days=7):
        ax.xaxis.set_major_locator(mdates.DayLocator())
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))
    elif time_span <= timedelta(days=30):
        ax.xaxis.set_major_locator(mdates.DayLocator(interval=2))
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))
    else:
        ax.xaxis.set_major_locator(mdates.WeekdayLocator())
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))

    ax.figure.autofmt_xdate()

def extract_temperature_series(api_data):
    dates = [entry['dt_txt'] for entry in api_data.get('list', [])]
    temps = [entry['main']['temp'] for entry in api_data.get('list', [])]
    return {'date': dates, 'temperature': temps}

def create_temperature_visualisation(weather_data, output_type='display', max_points=200, timezone='UTC'):
    df = preprocess_weather_data(weather_data['date'], weather_data['temperature'], max_points, timezone)

    fig, ax = plt.subplots(figsize=(10, 5))
    sns.lineplot(data=df, x='Date', y='Value', marker='o', ax=ax)
    ax.set_title(f"Temperature Trend ({timezone})")
    ax.set_xlabel("Date and Time")
    ax.set_ylabel("Temperature (°C)")
    ax.grid(True)

    format_time_axis(ax, df['Date'])
    fig.tight_layout()

    if output_type == 'figure':
        plt.close(fig)
        return fig
    else:
        plt.show()

# === CLI Interface ===

def main():
    print("=== Weather Forecast with Timezone-Aware Plots ===")
    location = input("Enter location (e.g., 'New York'): ")

    try:
        forecast_days = int(input("Enter number of forecast days (1–5): "))
        if not (1 <= forecast_days <= 5):
            raise ValueError
    except ValueError:
        print("[ERROR] Invalid number of days.")
        return

    units = input("Enter units (metric / imperial / standard): ").strip().lower()

    try:
        timezone = get_timezone_for_city(location)
        print(f"[INFO] Detected timezone for {location}: {timezone}")
    except Exception as e:
        print(f"[ERROR] Failed to detect timezone: {e}")
        timezone = "UTC"

    api_data = get_weather_data(location, forecast_days, units)
    if not api_data or "list" not in api_data:
        print("[ERROR] No weather data returned.")
        return

    weather_data = extract_temperature_series(api_data)
    create_temperature_visualisation(weather_data, timezone=timezone)

# === Entry Point ===

if __name__ == "__main__":
    main()


In [ ]:

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd
import random
from datetime import datetime, timedelta
import matplotlib.animation as animation

# ==========================
# CONFIGURATION
# ==========================
HISTORY_WINDOW_MINUTES = 5     # Show last 5 minutes of data
UPDATE_INTERVAL_MS = 1000      # Update every second
Y_AXIS_MIN = 15                # Temperature lower bound
Y_AXIS_MAX = 25                # Temperature upper bound

# ==========================
# SIMULATED TEMPERATURE DATA
# ==========================
def get_live_temperature_reading():
    """
    Simulate fetching a live temperature reading.
    Replace this function with a real API call for live data.
    """
    now = datetime.utcnow()
    temp = 20 + random.uniform(-2, 2)  # Simul


## 🤖 Natural Language Processing

In [ ]:
import re

# -----------------------
# Message Templates (i18n)
# -----------------------
MESSAGES = {
    'en': {
        'forecast': "Sure! Here's the weather forecast for {location} {time}.",
        'attribute': "I can tell you about the {attribute} in {location} {time}.",
        'sunny': "You want to know if it’ll be sunny in {location} {time}, right?",
        'default': "Let me check the weather in {location} {time}."
    },
    'fr': {
        'forecast': "Voici les prévisions météo pour {location} {time}.",
        'attribute': "Je peux vous parler de {attribute} à {location} {time}.",
        'sunny': "Vous voulez savoir s'il fera ensoleillé à {location} {time}, n'est-ce pas ?",
        'default': "Je vérifie la météo à {location} {time}."
    }
}

# -----------------------
# Patterns for Parsing
# -----------------------
TIME_PERIOD_PATTERNS = {
    'today': [r'\btoday\b', r"\baujourd'hui\b"],
    'tomorrow': [r'\btomorrow\b', r'\bdemain\b'],
    'week': [r'\bnext\s?7\s?days\b', r'\bweek\b', r'\bsemaine\b']
}

WEATHER_ATTRIBUTES = [
    'temperature', 'rain', 'precipitation', 'humidity',
    'wind', 'sunny', 'cloudy', 'forecast', 'neige', 'pluie'
]

# -----------------------
# Extraction Functions
# -----------------------
def extract_location(question: str) -> str:
    match = re.search(r'\bin ([a-zA-ZÀ-ÿ\s]+)', question)
    if match:
        return match.group(1).strip()

    tokens = question.split()
    for word in reversed(tokens):
        if word.lower() not in WEATHER_ATTRIBUTES:
            return word.strip(",.?")
    return "your area"


def extract_time_period(question: str) -> str:
    for label, patterns in TIME_PERIOD_PATTERNS.items():
        for pattern in patterns:
            if re.search(pattern, question):
                return label
    return 'today'


def extract_weather_attribute(question: str) -> str:
    for attr in WEATHER_ATTRIBUTES:
        if re.search(rf'\b{re.escape(attr)}\b', question):
            return attr
    return 'forecast'


def parse_weather_question(question: str) -> dict:
    q = question.lower()
    return {
        'location': extract_location(q),
        'time_period': extract_time_period(q),
        'weather_attribute': extract_weather_attribute(q)
    }

# -----------------------
# Localized Response
# -----------------------
def generate_localized_response(parsed: dict, lang: str = 'en') -> str:
    location = parsed.get('location', 'your area').title()
    time = parsed.get('time_period', 'today')
    attribute = parsed.get('weather_attribute', 'forecast')

    messages = MESSAGES.get(lang, MESSAGES['en'])

    if attribute == 'forecast':
        msg = messages['forecast']
    elif attribute in ['rain', 'snow', 'neige', 'pluie', 'wind', 'humidity', 'temperature']:
        msg = messages['attribute']
    elif attribute in ['sunny', 'cloudy']:
        msg = messages.get(attribute, messages['default'])
    else:
        msg = messages['default']

    return msg.format(location=location, time=time, attribute=attribute)

# -----------------------
# Main Chatbot Handler
# -----------------------
def handle_weather_question(question(question: str, lang: str = 'en') -> str:
    parsed = parse_weather_question(question)
    return generate_localized_response(parsed, lang)


## 🧭 User Interface

In [ ]:
import requests
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tkinter import messagebox
import customtkinter as ctk
from datetime import datetime, timedelta

# Constants
API_KEY = "your_api_key_here"
BASE_URL = "https://api.openweathermap.org/data/2.5/forecast"
UNITS = "metric"
INTERVALS_PER_DAY = 8  # 3-hour intervals

# Configurable rules for weather attributes
TIME_PERIOD_PATTERNS = {
    'today': [r'\btoday\b'],
    'tomorrow': [r'\btomorrow\b'],
    'week': [r'\bnext\s?7\s?days\b', r'\b7\s?days\b', r'\bweek\b']
}

WEATHER_ATTRIBUTES = [
    'temperature', 'rain', 'precipitation', 'humidity',
    'wind', 'sunny', 'cloudy', 'forecast'
]


def build_weather_query_params(location: str, forecast_days: int) -> dict:
    """Build query parameters for the weather API request."""
    return {
        "q": location,
        "cnt": forecast_days * INTERVALS_PER_DAY,
        "units": UNITS,
        "appid": API_KEY
    }


def fetch_weather_data(params: dict) -> dict:
    """Fetch weather data from the API."""
    try:
        response = requests.get(BASE_URL, params=params)
        response.raise_for_status()
        return response.json()
    except requests.RequestException as e:
        print(f"[ERROR] Failed to fetch weather data: {e}")
        return {}


def get_weather_data(location: str, forecast_days: int = 5) -> dict:
    """Fetch weather data and handle missing or 'unknown' values."""
    params = build_weather_query_params(location, forecast_days)
    data = fetch_weather_data(params)

    # Handling missing data (replace None with 'N/A')
    if data:
        for forecast in data['list']:
            forecast['main']['temp'] = forecast['main'].get('temp', 'N/A')
            forecast['rain'] = forecast.get('rain', {}).get('3h', 'N/A')

    return data


def preprocess_weather_data(dates, values, max_points=200):
    """Preprocess data for plotting by limiting points."""
    df = pd.DataFrame({
        'Date': pd.to_datetime(dates),
        'Value': values
    })
    # Limit the number of points to avoid cluttering the plot
    return df.tail(max_points)


def create_temperature_visualisation(weather_data, output_type='display'):
    """Create a temperature visualization."""
    dates = [forecast['dt_txt'] for forecast in weather_data['list']]
    temperatures = [forecast['main']['temp'] if forecast['main']['temp'] != 'N/A' else None for forecast in weather_data['list']]

    # Preprocess and plot
    df = preprocess_weather_data(dates, temperatures)
    df['Temperature'] = pd.to_numeric(df['Temperature'], errors='coerce')

    plt.figure(figsize=(10, 5))
    sns.lineplot(data=df, x='Date', y='Temperature', marker='o')
    plt.title('Temperature Trend')
    plt.xlabel('Date')
    plt.ylabel('Temperature (°C)')
    plt.xticks(rotation=45)
    plt.tight_layout()

    # Handle missing values: if a plot is incomplete, show a warning
    if df['Temperature'].isnull().any():
        plt.text(0.5, 0.5, 'Warning: Some data points are missing or unavailable',
                 ha='center', va='center', color='red', fontsize=14, alpha=0.7)

    if output_type == 'figure':
        fig = plt.gcf()
        plt.close(fig)
        return fig
    else:
        plt.show()


def create_precipitation_visualisation(weather_data, output_type='display'):
    """Create a precipitation visualization."""
    dates = [forecast['dt_txt'] for forecast in weather_data['list']]
    precipitations = [forecast['rain'] if forecast['rain'] != 'N/A' else None for forecast in weather_data['list']]

    # Preprocess and plot
    df = preprocess_weather_data(dates, precipitations)
    df['Precipitation'] = pd.to_numeric(df['Precipitation'], errors='coerce')

    plt.figure(figsize=(10, 5))
    sns.barplot(data=df, x='Date', y='Precipitation', color='skyblue')
    plt.title('Precipitation Forecast')
    plt.xlabel('Date')
    plt.ylabel('Precipitation (mm)')
    plt.xticks(rotation=45)
    plt.tight_layout()

    # Handle missing values: if a plot is incomplete, show a warning
    if df['Precipitation'].isnull().any():
        plt.text(0.5, 0.5, 'Warning: Some data points are missing or unavailable',
                 ha='center', va='center', color='red', fontsize=14, alpha=0.7)

    if output_type == 'figure':
        fig = plt.gcf()
        plt.close(fig)
        return fig
    else:
        plt.show()


# Tkinter-based App for User Interaction
def on_submit():
    """Function triggered on clicking the 'Submit' button."""
    loc = location_entry.get()
    try:
        days = int(days_entry.get())
        if days < 1 or days > 7:
            raise ValueError("Please enter a number between 1 and 7.")
    except ValueError as e:
        messagebox.showerror("Invalid Input", f"Invalid input for days: {str(e)}")
        return

    weather_data = get_weather_data(loc, days)

    # Check if any essential data is missing
    if not weather_data or not weather_data.get('list'):
        messagebox.showerror("Data Unavailable", "Sorry, the weather data is not available for this location.")
        return

    if visual_type.get() == "Temperature":
        create_temperature_visualisation(weather_data)
    else:
        create_precipitation_visualisation(weather_data)


def run_app():
    """Run the WeatherWise app with customtkinter."""
    app = ctk.CTk()
    app.geometry("400x300")
    app.title("WeatherWise")

    ctk.CTkLabel(app, text="WeatherWise", font=("Arial", 20)).pack(pady=10)

    location_entry = ctk.CTkEntry(app, placeholder_text="Enter location")
    location_entry.pack(pady=10)

    days_entry = ctk.CTkEntry(app, placeholder_text="Forecast days (1-7)")
    days_entry.pack(pady=10)

    visual_type = ctk.CTkOptionMenu(app, values=["Temperature", "Precipitation"])
    visual_type.set("Temperature")
    visual_type.pack(pady=10)

    submit_btn = ctk.CTkButton(app, text="Show Visualization", command=on_submit)
    submit_btn.pack(pady=20)

    app.mainloop()


if __name__ == "__main__":
    run_app()


ModuleNotFoundError: No module named 'customtkinter'

## 🧩 Main Application Logic

In [2]:
# 🧪 Optional packages — uncomment if needed in Colab or JupyterHub
!pip install fetch-my-weather
!pip install hands-on-ai
!pip install pyinputplus
!pip install customtkinter
!pip install requests
!pip install timezonefinder geopy
!pip install langdetect


import os

os.environ['HANDS_ON_AI_SERVER'] = 'http://ollama.serveur.au'
os.environ['HANDS_ON_AI_MODEL'] = 'granite3.2'
os.environ['HANDS_ON_AI_API_KEY'] = input('Enter your API key: ')

import requests
import matplotlib.pyplot as plt
import pyinputplus as pyip
import pandas as pd
from datetime import datetime, timedelta

import seaborn as sns
# ✅ Import after installing (if needed)
from fetch_my_weather import get_weather
from hands_on_ai.chat import get_response

# Add any other setup code here

import requests
from typing import Dict

# === Configuration ===
API_KEY = "your_api_key_here"  # Replace with your actual API key
BASE_URL = "https://api.openweathermap.org/data/2.5/forecast"
INTERVALS_PER_DAY = 8  # 3-hour intervals

# === Core Weather Functions ===

def build_weather_query_params(location: str, forecast_days: int, units: str) -> Dict[str, str]:
    """Build query parameters for the weather API request."""
    return {
        "q": location,
        "cnt": forecast_days * INTERVALS_PER_DAY,
        "units": units,  # 'metric', 'imperial', or 'standard'
        "appid": API_KEY
    }

def fetch_weather_data(params: dict) -> dict:
    """Fetch weather data from the API."""
    try:
        response = requests.get(BASE_URL, params=params)
        response.raise_for_status()
        return response.json()
    except requests.RequestException as e:
        print(f"[ERROR] Failed to fetch weather data: {e}")
        return {}

def get_weather_data(location: str, forecast_days: int = 5, units: str = "metric") -> dict:
    """Main function to get weather data for a given location and forecast period."""
    if units not in ["standard", "metric", "imperial"]:
        print(f"[WARNING] Invalid units '{units}' provided. Defaulting to 'metric'.")
        units = "metric"

    params = build_weather_query_params(location, forecast_days, units)
    return fetch_weather_data(params)

# === Display Logic ===

def display_weather(data: dict, units: str):
    """Format and display the weather data."""
    if not data or "list" not in data:
        print("No weather data available.")
        return

    city = data.get("city", {}).get("name", "Unknown location")
    country = data.get("city", {}).get("country", "")
    print(f"\n📍 Weather Forecast for {city}, {country}:\n")

    temp_unit = "°C" if units == "metric" else "°F" if units == "imperial" else "K"
    wind_unit = "m/s" if units in ["metric", "standard"] else "mph"

    for forecast in data["list"]:
        dt_txt = forecast["dt_txt"]
        temp = forecast["main"]["temp"]
        weather = forecast["weather"][0]["description"].capitalize()
        wind = forecast["wind"]["speed"]

        print(f"{dt_txt} | 🌡 Temp: {temp}{temp_unit} | 🌬 Wind: {wind} {wind_unit} | 🌥 {weather}")

# === CLI Interface ===

def main():
    print("=== Weather Forecast CLI ===")
    location = input("Enter location (e.g., London): ")
    try:
        days = int(input("Enter number of forecast days (1-5): "))
        if not (1 <= days <= 5):
            raise ValueError
    except ValueError:
        print("[ERROR] Please enter a valid number of days (1–5).")
        return

    units = input("Enter units (metric / imperial / standard): ").strip().lower()

    data = get_weather_data(location, days, units)
    display_weather(data, units)

# === Run Program ===

if __name__ == "__main__":
    main()

# === Imports ===
import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates
from datetime import timedelta
from geopy.geocoders import Nominatim
from timezonefinder import TimezoneFinder
from zoneinfo import ZoneInfo  # Requires Python 3.9+

# === Configuration ===
API_KEY = "your_api_key_here"  # Replace with your OpenWeatherMap API key
BASE_URL = "https://api.openweathermap.org/data/2.5/forecast"
INTERVALS_PER_DAY = 8

# === Timezone Utilities ===

def get_lat_lon_from_city(city_name):
    geolocator = Nominatim(user_agent="weather-app")
    location = geolocator.geocode(city_name)
    if location:
        return location.latitude, location.longitude
    else:
        raise ValueError(f"Could not find location for city: {city_name}")

def get_timezone_from_latlon(lat, lon):
    tf = TimezoneFinder()
    timezone = tf.timezone_at(lat=lat, lng=lon)
    if timezone:
        return timezone
    else:
        raise ValueError("Could not determine timezone.")

def get_timezone_for_city(city_name):
    lat, lon = get_lat_lon_from_city(city_name)
    return get_timezone_from_latlon(lat, lon)

def convert_to_local_time(dates_utc, timezone_str):
    return pd.to_datetime(dates_utc).dt.tz_localize('UTC').dt.tz_convert(ZoneInfo(timezone_str))

# === Weather Data Fetching ===

def build_weather_query_params(location: str, forecast_days: int, units: str):
    return {
        "q": location,
        "cnt": forecast_days * INTERVALS_PER_DAY,
        "units": units,
        "appid": API_KEY
    }

def fetch_weather_data(params: dict):
    try:
        response = requests.get(BASE_URL, params=params)
        response.raise_for_status()
        return response.json()
    except requests.RequestException as e:
        print(f"[ERROR] Failed to fetch weather data: {e}")
        return {}

def get_weather_data(location: str, forecast_days: int = 5, units: str = "metric"):
    if units not in ["metric", "imperial", "standard"]:
        print(f"[WARNING] Invalid units '{units}' provided. Defaulting to 'metric'.")
        units = "metric"

    params = build_weather_query_params(location, forecast_days, units)
    return fetch_weather_data(params)

# === Preprocessing and Plotting ===

def preprocess_weather_data(dates, values, max_points=200, timezone="UTC"):
    df = pd.DataFrame({
        'Date': pd.to_datetime(dates),
        'Value': values
    })
    df['Date'] = convert_to_local_time(df['Date'], timezone)
    if len(df) > max_points:
        df = df.iloc[::len(df) // max_points]
    return df

def format_time_axis(ax, dates):
    if isinstance(dates, list):
        dates = pd.to_datetime(dates)

    time_span = dates.max() - dates.min()

    if time_span <= timedelta(hours=12):
        ax.xaxis.set_major_locator(mdates.HourLocator(interval=1))
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
    elif time_span <= timedelta(days=1):
        ax.xaxis.set_major_locator(mdates.HourLocator(interval=3))
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
    elif time_span <= timedelta(days=7):
        ax.xaxis.set_major_locator(mdates.DayLocator())
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))
    elif time_span <= timedelta(days=30):
        ax.xaxis.set_major_locator(mdates.DayLocator(interval=2))
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))
    else:
        ax.xaxis.set_major_locator(mdates.WeekdayLocator())
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))

    ax.figure.autofmt_xdate()

def extract_temperature_series(api_data):
    dates = [entry['dt_txt'] for entry in api_data.get('list', [])]
    temps = [entry['main']['temp'] for entry in api_data.get('list', [])]
    return {'date': dates, 'temperature': temps}

def create_temperature_visualisation(weather_data, output_type='display', max_points=200, timezone='UTC'):
    df = preprocess_weather_data(weather_data['date'], weather_data['temperature'], max_points, timezone)

    fig, ax = plt.subplots(figsize=(10, 5))
    sns.lineplot(data=df, x='Date', y='Value', marker='o', ax=ax)
    ax.set_title(f"Temperature Trend ({timezone})")
    ax.set_xlabel("Date and Time")
    ax.set_ylabel("Temperature (°C)")
    ax.grid(True)

    format_time_axis(ax, df['Date'])
    fig.tight_layout()

    if output_type == 'figure':
        plt.close(fig)
        return fig
    else:
        plt.show()

# === CLI Interface ===

def main():
    print("=== Weather Forecast with Timezone-Aware Plots ===")
    location = input("Enter location (e.g., 'New York'): ")

    try:
        forecast_days = int(input("Enter number of forecast days (1–5): "))
        if not (1 <= forecast_days <= 5):
            raise ValueError
    except ValueError:
        print("[ERROR] Invalid number of days.")
        return

    units = input("Enter units (metric / imperial / standard): ").strip().lower()

    try:
        timezone = get_timezone_for_city(location)
        print(f"[INFO] Detected timezone for {location}: {timezone}")
    except Exception as e:
        print(f"[ERROR] Failed to detect timezone: {e}")
        timezone = "UTC"

    api_data = get_weather_data(location, forecast_days, units)
    if not api_data or "list" not in api_data:
        print("[ERROR] No weather data returned.")
        return

    weather_data = extract_temperature_series(api_data)
    create_temperature_visualisation(weather_data, timezone=timezone)

# === Entry Point ===

if __name__ == "__main__":
    main()

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd
import random
from datetime import datetime, timedelta
import matplotlib.animation as animation

# ==========================
# CONFIGURATION
# ==========================
HISTORY_WINDOW_MINUTES = 5     # Show last 5 minutes of data
UPDATE_INTERVAL_MS = 1000      # Update every second
Y_AXIS_MIN = 15                # Temperature lower bound
Y_AXIS_MAX = 25                # Temperature upper bound

# ==========================
# SIMULATED TEMPERATURE DATA
# ==========================
def get_live_temperature_reading():
    """
    Simulate fetching a live temperature reading.
    Replace this function with a real API call for live data.
    """
    now = datetime.utcnow()
    temp = 20 + random.uniform(-2, 2)  # Simul

import re

# -----------------------
# Message Templates (i18n)
# -----------------------
MESSAGES = {
    'en': {
        'forecast': "Sure! Here's the weather forecast for {location} {time}.",
        'attribute': "I can tell you about the {attribute} in {location} {time}.",
        'sunny': "You want to know if it’ll be sunny in {location} {time}, right?",
        'default': "Let me check the weather in {location} {time}."
    },
    'fr': {
        'forecast': "Voici les prévisions météo pour {location} {time}.",
        'attribute': "Je peux vous parler de {attribute} à {location} {time}.",
        'sunny': "Vous voulez savoir s'il fera ensoleillé à {location} {time}, n'est-ce pas ?",
        'default': "Je vérifie la météo à {location} {time}."
    }
}

# -----------------------
# Patterns for Parsing
# -----------------------
TIME_PERIOD_PATTERNS = {
    'today': [r'\btoday\b', r"\baujourd'hui\b"],
    'tomorrow': [r'\btomorrow\b', r'\bdemain\b'],
    'week': [r'\bnext\s?7\s?days\b', r'\bweek\b', r'\bsemaine\b']
}

WEATHER_ATTRIBUTES = [
    'temperature', 'rain', 'precipitation', 'humidity',
    'wind', 'sunny', 'cloudy', 'forecast', 'neige', 'pluie'
]

# -----------------------
# Extraction Functions
# -----------------------
def extract_location(question: str) -> str:
    match = re.search(r'\bin ([a-zA-ZÀ-ÿ\s]+)', question)
    if match:
        return match.group(1).strip()

    tokens = question.split()
    for word in reversed(tokens):
        if word.lower() not in WEATHER_ATTRIBUTES:
            return word.strip(",.?")
    return "your area"


def extract_time_period(question: str) -> str:
    for label, patterns in TIME_PERIOD_PATTERNS.items():
        for pattern in patterns:
            if re.search(pattern, question):
                return label
    return 'today'


def extract_weather_attribute(question: str) -> str:
    for attr in WEATHER_ATTRIBUTES:
        if re.search(rf'\b{re.escape(attr)}\b', question):
            return attr
    return 'forecast'


def parse_weather_question(question: str) -> dict:
    q = question.lower()
    return {
        'location': extract_location(q),
        'time_period': extract_time_period(q),
        'weather_attribute': extract_weather_attribute(q)
    }

# -----------------------
# Localized Response
# -----------------------
def generate_localized_response(parsed: dict, lang: str = 'en') -> str:
    location = parsed.get('location', 'your area').title()
    time = parsed.get('time_period', 'today')
    attribute = parsed.get('weather_attribute', 'forecast')

    messages = MESSAGES.get(lang, MESSAGES['en'])

    if attribute == 'forecast':
        msg = messages['forecast']
    elif attribute in ['rain', 'snow', 'neige', 'pluie', 'wind', 'humidity', 'temperature']:
        msg = messages['attribute']
    elif attribute in ['sunny', 'cloudy']:
        msg = messages.get(attribute, messages['default'])
    else:
        msg = messages['default']

    return msg.format(location=location, time=time, attribute=attribute)

# -----------------------
# Main Chatbot Handler
# -----------------------
def handle_weather_question(question: str, lang: str = 'en') -> str:
    parsed = parse_weather_question(question)
    return generate_localized_response(parsed, lang)

import requests
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tkinter import messagebox
import customtkinter as ctk
from datetime import datetime, timedelta

# Constants
API_KEY = "your_api_key_here"
BASE_URL = "https://api.openweathermap.org/data/2.5/forecast"
UNITS = "metric"
INTERVALS_PER_DAY = 8  # 3-hour intervals

# Configurable rules for weather attributes
TIME_PERIOD_PATTERNS = {
    'today': [r'\btoday\b'],
    'tomorrow': [r'\btomorrow\b'],
    'week': [r'\bnext\s?7\s?days\b', r'\b7\s?days\b', r'\bweek\b']
}

WEATHER_ATTRIBUTES = [
    'temperature', 'rain', 'precipitation', 'humidity',
    'wind', 'sunny', 'cloudy', 'forecast'
]


def build_weather_query_params(location: str, forecast_days: int) -> dict:
    """Build query parameters for the weather API request."""
    return {
        "q": location,
        "cnt": forecast_days * INTERVALS_PER_DAY,
        "units": UNITS,
        "appid": API_KEY
    }


def fetch_weather_data(params: dict) -> dict:
    """Fetch weather data from the API."""
    try:
        response = requests.get(BASE_URL, params=params)
        response.raise_for_status()
        return response.json()
    except requests.RequestException as e:
        print(f"[ERROR] Failed to fetch weather data: {e}")
        return {}


def get_weather_data(location: str, forecast_days: int = 5) -> dict:
    """Fetch weather data and handle missing or 'unknown' values."""
    params = build_weather_query_params(location, forecast_days)
    data = fetch_weather_data(params)

    # Handling missing data (replace None with 'N/A')
    if data:
        for forecast in data['list']:
            forecast['main']['temp'] = forecast['main'].get('temp', 'N/A')
            forecast['rain'] = forecast.get('rain', {}).get('3h', 'N/A')

    return data


def preprocess_weather_data(dates, values, max_points=200):
    """Preprocess data for plotting by limiting points."""
    df = pd.DataFrame({
        'Date': pd.to_datetime(dates),
        'Value': values
    })
    # Limit the number of points to avoid cluttering the plot
    return df.tail(max_points)


def create_temperature_visualisation(weather_data, output_type='display'):
    """Create a temperature visualization."""
    dates = [forecast['dt_txt'] for forecast in weather_data['list']]
    temperatures = [forecast['main']['temp'] if forecast['main']['temp'] != 'N/A' else None for forecast in weather_data['list']]

    # Preprocess and plot
    df = preprocess_weather_data(dates, temperatures)
    df['Temperature'] = pd.to_numeric(df['Temperature'], errors='coerce')

    plt.figure(figsize=(10, 5))
    sns.lineplot(data=df, x='Date', y='Temperature', marker='o')
    plt.title('Temperature Trend')
    plt.xlabel('Date')
    plt.ylabel('Temperature (°C)')
    plt.xticks(rotation=45)
    plt.tight_layout()

    # Handle missing values: if a plot is incomplete, show a warning
    if df['Temperature'].isnull().any():
        plt.text(0.5, 0.5, 'Warning: Some data points are missing or unavailable',
                 ha='center', va='center', color='red', fontsize=14, alpha=0.7)

    if output_type == 'figure':
        fig = plt.gcf()
        plt.close(fig)
        return fig
    else:
        plt.show()


def create_precipitation_visualisation(weather_data, output_type='display'):
    """Create a precipitation visualization."""
    dates = [forecast['dt_txt'] for forecast in weather_data['list']]
    precipitations = [forecast['rain'] if forecast['rain'] != 'N/A' else None for forecast in weather_data['list']]

    # Preprocess and plot
    df = preprocess_weather_data(dates, precipitations)
    df['Precipitation'] = pd.to_numeric(df['Precipitation'], errors='coerce')

    plt.figure(figsize=(10, 5))
    sns.barplot(data=df, x='Date', y='Precipitation', color='skyblue')
    plt.title('Precipitation Forecast')
    plt.xlabel('Date')
    plt.ylabel('Precipitation (mm)')
    plt.xticks(rotation=45)
    plt.tight_layout()

    # Handle missing values: if a plot is incomplete, show a warning
    if df['Precipitation'].isnull().any():
        plt.text(0.5, 0.5, 'Warning: Some data points are missing or unavailable',
                 ha='center', va='center', color='red', fontsize=14, alpha=0.7)

    if output_type == 'figure':
        fig = plt.gcf()
        plt.close(fig)
        return fig
    else:
        plt.show()


# Tkinter-based App for User Interaction
def on_submit():
    """Function triggered on clicking the 'Submit' button."""
    loc = location_entry.get()
    try:
        days = int(days_entry.get())
        if days < 1 or days > 7:
            raise ValueError("Please enter a number between 1 and 7.")
    except ValueError as e:
        messagebox.showerror("Invalid Input", f"Invalid input for days: {str(e)}")
        return

    weather_data = get_weather_data(loc, days)

    # Check if any essential data is missing
    if not weather_data or not weather_data.get('list'):
        messagebox.showerror("Data Unavailable", "Sorry, the weather data is not available for this location.")
        return

    if visual_type.get() == "Temperature":
        create_temperature_visualisation(weather_data)
    else:
        create_precipitation_visualisation(weather_data)


def run_app():
    """Run the WeatherWise app with customtkinter."""
    app = ctk.CTk()
    app.geometry("400x300")
    app.title("WeatherWise")

    ctk.CTkLabel(app, text="WeatherWise", font=("Arial", 20)).pack(pady=10)

    location_entry = ctk.CTkEntry(app, placeholder_text="Enter location")
    location_entry.pack(pady=10)

    days_entry = ctk.CTkEntry(app, placeholder_text="Forecast days (1-7)")
    days_entry.pack(pady=10)

    visual_type = ctk.CTkOptionMenu(app, values=["Temperature", "Precipitation"])
    visual_type.set("Temperature")
    visual_type.pack(pady=10)

    submit_btn = ctk.CTkButton(app, text="Show Visualization", command=on_submit)
    submit_btn.pack(pady=20)

    app.mainloop()


if __name__ == "__main__":
    run_app()



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 86.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.9/69.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.5/80.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 459.8/459.8 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 84.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.8/454.8 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 90.7 MB/s eta 0:00:00
  Attempting uninstall: fastcore
    Found existing in

TclError: no display name and no $DISPLAY environment variable

## 🧪 Testing and Examples

In [ ]:
# Include sample input/output for each function

## 🗂️ AI Prompting Log (Optional)
Add markdown cells here summarising prompts used or link to AI conversations in the `ai-conversations/` folder.